In [1]:
!wget https://raw.githubusercontent.com/Tinver93/hse22_hw1/main/data/out_gapClosed.fa
!wget https://raw.githubusercontent.com/Tinver93/hse22_hw2/main/data/gms2.lst
!wget https://raw.githubusercontent.com/Tinver93/hse22_hw2/main/data/proteins.fasta
!wget https://raw.githubusercontent.com/Tinver93/hse22_hw2/main/data/scaffolds.hits_from_MIL_1.txt
!wget https://raw.githubusercontent.com/Tinver93/hse22_hw2/main/data/scaffolds.hits_from_SwissProt.txt

--2022-10-19 15:09:53--  https://raw.githubusercontent.com/Tinver93/hse22_hw1/main/data/out_gapClosed.fa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3973489 (3.8M) [text/plain]
Saving to: ‘out_gapClosed.fa’

out_gapClosed.fa    100%[===================>]   3.79M  --.-KB/s    in 0.07s   

2022-10-19 15:09:54 (52.7 MB/s) - ‘out_gapClosed.fa’ saved [3973489/3973489]

--2022-10-19 15:09:54--  https://raw.githubusercontent.com/Tinver93/hse22_hw2/main/data/gms2.lst
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221729 (217K) [text/

In [2]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=\${PATH}:/root/edirect" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
y
OK, done.

To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [3]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

In [4]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 5.2 MB/s 


In [18]:
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
from datetime import datetime as dt
import pandas as pd

In [19]:
scafs = dict()
for elem in SeqIO.parse("out_gapClosed.fa", "fasta"):
  elem.annotations['molecule_type'] = 'DNA'
  elem.annotations['date'] = dt.now().strftime("%d-%b-%Y").upper()
  elem.annotations['data_file_division'] = 'BCT'
  
  scafs[elem.id] = elem

genes = dict()
for gene in SeqIO.parse("proteins.fasta", "fasta"):
  desc = gene.description.split(' ')
  scaf = desc[1]
  start, end = int(desc[2]), int(desc[3])
  if desc[4] == '+':
    strand = 1
  else:
    strand = -1
  
  feat = SeqFeature(FeatureLocation(start, end, strand=strand), type="CDS")
  feat.qualifiers['locus_tag'] = [desc[0]]
  feat.qualifiers['translation'] = [gene.seq]
  scafs[scaf].features.append(feat)
  
  genes[desc[0]] = feat

feat.qualifiers

OrderedDict([('locus_tag', ['3618']),
             ('translation',
              [Seq('GLPLYRMETIFKRMGIHLPRNTLANWMIKSSECLQPLYNLLNDQLLESGYLHMD...DDY')])])

In [41]:
mil1 = dict()
for feat in SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank").features:
  if 'protein_id' not in feat.qualifiers:
    continue
  if 'product' not in feat.qualifiers:
    continue
  
  mil1[feat.qualifiers['protein_id'][0]] = feat.qualifiers['product'][0]

names = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
mil1h = pd.read_csv('scaffolds.hits_from_MIL_1.txt', sep='\t', header=None, names=names)
mil1h

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,1,lcl|HF680312.1_prot_CCU71653.1_1204,98.990,99,1,0,1,99,1,99,8.080000e-68,196.0
1,1,lcl|HF680312.1_prot_CCU72283.1_1834,97.980,99,2,0,1,99,1,99,2.290000e-67,194.0
2,1,lcl|HF680312.1_prot_CCU71933.1_1484,97.980,99,2,0,1,99,1,99,4.150000e-66,191.0
3,1,lcl|HF680312.1_prot_CCU71592.1_1143,97.980,99,2,0,1,99,1,99,4.150000e-66,191.0
4,1,lcl|HF680312.1_prot_CCU71865.1_1416,96.774,62,2,0,1,62,1,62,2.150000e-42,130.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10531,3618,lcl|HF680312.1_prot_CCU73517.1_3068,96.552,116,4,0,1,116,216,331,4.090000e-80,241.0
10532,3618,lcl|HF680312.1_prot_CCU73713.1_3264,96.552,116,4,0,1,116,216,331,5.770000e-80,241.0
10533,3618,lcl|HF680312.1_prot_CCU73876.1_3427,96.552,116,4,0,1,116,223,338,8.800000e-80,241.0
10534,3618,lcl|HF680312.1_prot_CCU73197.1_2748,96.552,116,4,0,1,116,223,338,8.800000e-80,241.0


In [42]:
hits = mil1h[mil1h['sseqid'].str.contains("CCU")].sort_values('bitscore', ascending=False).drop_duplicates('qseqid')
hits

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
2832,1024,lcl|HF680312.1_prot_CCU73896.1_3447,98.979,2253,23,0,1,2253,1,2253,0.000000e+00,4296.0
4239,1486,lcl|HF680312.1_prot_CCU70724.1_275,97.870,2113,43,1,7,2119,1,2111,0.000000e+00,4158.0
9141,3216,lcl|HF680312.1_prot_CCU71900.1_1451,99.878,1640,2,0,1,1640,1,1640,0.000000e+00,3406.0
2609,947,lcl|HF680312.1_prot_CCU73861.1_3412,99.214,1654,13,0,1,1654,1,1654,0.000000e+00,3372.0
9920,3493,lcl|HF680312.1_prot_CCU71621.1_1172,99.690,1611,5,0,1,1611,1,1611,0.000000e+00,3338.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9910,3485,lcl|HF680312.1_prot_CCU71629.1_1180,61.224,49,8,1,2,39,139,187,1.280000e-11,54.7
10389,3593,lcl|HF680312.1_prot_CCU70460.1_11,29.592,98,63,2,16,112,4,96,1.250000e-11,53.5
4799,1679,lcl|HF680312.1_prot_CCU72336.1_1887,29.592,98,63,2,16,112,4,96,1.250000e-11,53.5
8051,2814,lcl|HF680312.1_prot_CCU73193.1_2744,29.592,98,63,2,16,112,4,96,1.250000e-11,53.5


In [44]:
for i, hit in hits.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('_')[2]
  gene.qualifiers['product'] = [mil1[match]]

In [33]:
!wget -nc https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
!gzip -d uniprot_sprot.dat.gz

--2022-10-19 15:44:53--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 642093634 (612M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz’

uniprot_sprot.dat.g 100%[===================>] 612.35M  18.4MB/s    in 17s     

2022-10-19 15:45:10 (35.8 MB/s) - ‘uniprot_sprot.dat.gz’ saved [642093634/642093634]



In [45]:
!grep '^ID\|^DE   RecName: Full=' uniprot_sprot.dat > SwissProt_names.txt

In [46]:
previd = None
swissgenes = dict()
for line in open('SwissProt_names.txt'):
  if line.startswith('ID'):
    previd = line.split()[1]
  if line.startswith('DE'):
    swissgenes[previd] = line.split('=')[1][:-2]

swisshits_raw = pd.read_csv('scaffolds.hits_from_SwissProt.txt', sep='\t', header=None, names=names)
swisshits = swisshits_raw.sort_values('bitscore', ascending=False).drop_duplicates('qseqid')

for i, hit in swisshits.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('|')[-1]
  gene.qualifiers['product'] = [swissgenes[match]]

SeqIO.write(scafs.values(), "GENOME.gbk", "genbank")

75